# Project 2: [Analysez des données nutritionnelles](https://openclassrooms.com/projects/analysez-des-donnees-nutritionnelles)

Notebook tips:  
* keyborad shortcuts: Command mode (Esc) / H
* markdown [basic shortcuts](https://fr.wikipedia.org/wiki/Markdown)

## Data Cleaning Notebook

Goal: transform the raw dataset into an analysis-friendly one.

* Download the data [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip).
* Find some info [here](https://world.openfoodfacts.org/data/data-fields.txt).

> Comme vous le voyez, les champs sont séparés en quatre grandes sections :

> * Les informations générales sur la fiche du produit : nom, date de modification, etc.
> * Un ensemble de tags : catégorie du produit, localisation, origine, etc.
> * Les ingrédients composant les produits et leurs additifs éventuels
> * Des informations nutritionnelles : quantité en grammes d’un nutriment pour 100 grammes du produit

_General imports_

In [1]:
import os
HOME = os.path.expanduser('~/')
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

Let's import the data (assuming they have been downloaded to `Downloads/` and unzipped).  
Note that the original `.zip` file contains two junk files which prevent from using `pd.read_csv` directly on the url.

In [7]:
df = pd.read_csv(HOME+'/Downloads/fr.openfoodfacts.org.products.csv',
                 nrows=None,
                 low_memory=False, # to avoid type mixing due to batch reading
                 sep='\t')
print('loaded df with shape {}'.format(df.shape))

loaded df with shape (320772, 162)


In [8]:
df.sample(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
220718,3272872003909,http://world-fr.openfoodfacts.org/produit/3272...,tacite,1484400789,2017-01-14T13:33:09Z,1484905041,2017-01-20T09:37:21Z,Cuisse de poulet jaune S/V x2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110661,0099482452735,http://world-fr.openfoodfacts.org/produit/0099...,usda-ndb-import,1489062839,2017-03-09T12:33:59Z,1489062839,2017-03-09T12:33:59Z,Organic Cannellini Beans,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-7.0,-7.0,NaN,NaN
228049,3350033383692,http://world-fr.openfoodfacts.org/produit/3350...,openfoodfacts-contributors,1477678767,2016-10-28T18:19:27Z,1477682861,2016-10-28T19:27:41Z,Falafels,NaN,150 g,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.0,NaN,NaN
266761,4600384006634,http://world-fr.openfoodfacts.org/produit/4600...,openfoodfacts-contributors,1487751795,2017-02-22T08:23:15Z,1487751953,2017-02-22T08:25:53Z,Nesquik Opti-Start,NaN,250 г,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301582,8001120725103,http://world-fr.openfoodfacts.org/produit/8001...,openfoodfacts-contributors,1437840695,2015-07-25T16:11:35Z,1437840719,2015-07-25T16:11:59Z,Mozzarella di Bufala Campana,NaN,400g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Start by removing apriori useless variables. To do this, read variables descriptions and/or iterate looking at sample rows to see what each variable is about and/or the number of filled observations...

In [74]:
var_ID = tuple(np.sort((
       'code',                              # may be useful for joining...
       'product_name',
       'brands_tags', 'categories_fr',  # _fr are tags too
       'serving_size',
       'main_category_fr',
)))

var_content = tuple(np.sort((
       'ingredients_text', 'traces_fr', 'additives_n',
       'additives_fr', 'ingredients_from_palm_oil_n',
       'ingredients_from_palm_oil_tags',
       'nutrition_grade_fr',
)))

var_basicPer100g = tuple(np.sort((
        'calcium_100g', 'carbohydrates_100g', 'cholesterol_100g',
        'energy_100g', 'fat_100g', 'fiber_100g', 'proteins_100g', 'salt_100g',
        'saturated-fat_100g', 'sodium_100g', 'sugars_100g', 'trans-fat_100g',   
)))

var_allPer100g = tuple(np.sort((
       'energy_100g', 'energy-from-fat_100g', 'fat_100g', 'saturated-fat_100g',
       'butyric-acid_100g', 'caproic-acid_100g', 'caprylic-acid_100g',
       'capric-acid_100g', 'lauric-acid_100g', 'myristic-acid_100g',
       'palmitic-acid_100g', 'stearic-acid_100g', 'arachidic-acid_100g',
       'behenic-acid_100g', 'lignoceric-acid_100g', 'cerotic-acid_100g',
       'montanic-acid_100g', 'melissic-acid_100g',
       'monounsaturated-fat_100g', 'polyunsaturated-fat_100g',
       'omega-3-fat_100g', 'alpha-linolenic-acid_100g',
       'eicosapentaenoic-acid_100g', 'docosahexaenoic-acid_100g',
       'omega-6-fat_100g', 'linoleic-acid_100g', 'arachidonic-acid_100g',
       'gamma-linolenic-acid_100g', 'dihomo-gamma-linolenic-acid_100g',
       'omega-9-fat_100g', 'oleic-acid_100g', 'elaidic-acid_100g',
       'gondoic-acid_100g', 'mead-acid_100g', 'erucic-acid_100g',
       'nervonic-acid_100g', 'trans-fat_100g', 'cholesterol_100g',
       'carbohydrates_100g', 'sugars_100g', 'sucrose_100g', 'glucose_100g',
       'fructose_100g', 'lactose_100g', 'maltose_100g',
       'maltodextrins_100g', 'starch_100g', 'polyols_100g', 'fiber_100g',
       'proteins_100g', 'casein_100g', 'serum-proteins_100g',
       'nucleotides_100g', 'salt_100g', 'sodium_100g', 'alcohol_100g',
       'vitamin-a_100g', 'beta-carotene_100g', 'vitamin-d_100g',
       'vitamin-e_100g', 'vitamin-k_100g', 'vitamin-c_100g',
       'vitamin-b1_100g', 'vitamin-b2_100g', 'vitamin-pp_100g',
       'vitamin-b6_100g', 'vitamin-b9_100g', 'folates_100g',
       'vitamin-b12_100g', 'biotin_100g', 'pantothenic-acid_100g',
       'silica_100g', 'bicarbonate_100g', 'potassium_100g',
       'chloride_100g', 'calcium_100g', 'phosphorus_100g', 'iron_100g',
       'magnesium_100g', 'zinc_100g', 'copper_100g', 'manganese_100g',
       'fluoride_100g', 'selenium_100g', 'chromium_100g',
       'molybdenum_100g', 'iodine_100g', 'caffeine_100g', 'taurine_100g',
       'ph_100g', 'fruits-vegetables-nuts_100g',
       'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g',
       'carbon-footprint_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g', 'glycemic-index_100g', 'water-hardness_100g',
)))

# df = df.loc[:,var_ID+var_content+var_allPer100g]
df = df.loc[:,var_ID+var_content+var_basicPer100g] # start with basic _per100g data (other are poorly populated anyway)

In [75]:
df.sample(5)

,brands_tags,categories_fr,code,main_category_fr,product_name,serving_size,additives_fr,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil_tags,...,cholesterol_100g,energy_100g,fat_100g,fiber_100g,proteins_100g,salt_100g,saturated-fat_100g,sodium_100g,sugars_100g,trans-fat_100g
50661,simms,NaN,0041498167097,NaN,Beef Jerky,28 g (1 ONZ),"E330 - Acide citrique,E250 - Nitrite de sodium",2.0,0.0,NaN,...,0.054,1197.0,3.57,0.00,39.29,5.44322,0.00,2.143,21.43,0.0
178244,linessa,"Viandes,Charcuteries,Jambons,Jambons crus",20150952,Jambons crus,Jambon cru légèrement fumé,NaN,"E301 - Ascorbate de sodium,E250 - Nitrite de s...",3.0,0.0,NaN,...,NaN,563.0,3.00,0.01,26.00,5.08000,1.20,2.000,0.40,NaN
66732,el-bubble,NaN,0059642003719,NaN,"El Bubble || Original Bubble Gum, Orange, Grap...",20 g (1 PIECE),"E422 - Glycérol,E322 - Lécithines,E129 - Rouge...",7.0,0.0,NaN,...,NaN,1464.0,0.00,NaN,0.00,0.00000,NaN,0.000,75.00,NaN
90511,bob-evans,NaN,0075900005158,NaN,Pork Sausage Gravy,128 g (0.5 cup),"E375 - Acide nicotinique,E101 - Riboflavine,E3...",8.0,0.0,NaN,...,0.012,490.0,7.81,0.00,3.91,1.17094,3.12,0.461,0.78,0.0
109786,365-organic,en:Peanut-sauce,0099482404819,en:Peanut-sauce,Organic peanut sauce,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
print('percentage not NaN per variable:')
for v in df:
    print('{:>50}: {}'.format(v,int(100*df[v].count()/df.shape[0])))

percentage not NaN per variable:
                                       brands_tags: 91
                                     categories_fr: 26
                                              code: 99
                                  main_category_fr: 26
                                      product_name: 94
                                      serving_size: 65
                                      additives_fr: 48
                                       additives_n: 77
                       ingredients_from_palm_oil_n: 77
                    ingredients_from_palm_oil_tags: 1
                                  ingredients_text: 77
                                nutrition_grade_fr: 68
                                         traces_fr: 7
                                      calcium_100g: 43
                                carbohydrates_100g: 75
                                  cholesterol_100g: 44
                                       energy_100g: 81
                                  

See what some variables look like:

In [77]:
df[['ingredients_text','traces_fr']].sample(5)

,ingredients_text,traces_fr
155471,"Dark chocolate (chocolate liquor, sugar, cocoa...",NaN
206509,"Jus de soja** 97 % (eau, graines de _soja**_ 7...",NaN
32000,"Cultured grade a reduced fat milk, sugar, natu...",NaN
134823,"Beef, water, salt, corn syrup solids, citric a...",NaN
81657,Grade a milk & vitamin d3.,NaN


In [78]:
df[['additives_fr','additives_n']].sample(5)

,additives_fr,additives_n
65025,"E341 - Phosphate de calcium d'hydrogène,E339ii...",3.0
306857,E270 - Acide lactique,1.0
172441,E300 - Acide ascorbique,1.0
105189,"E440 - Pectines,E330 - Acide citrique,E300 - A...",5.0
274473,NaN,NaN


In [79]:
df[['ingredients_from_palm_oil_tags','ingredients_from_palm_oil_n']].sample(5)

,ingredients_from_palm_oil_tags,ingredients_from_palm_oil_n
306179,NaN,NaN
222179,NaN,NaN
259128,NaN,0.0
115908,NaN,0.0
190830,NaN,NaN


In [80]:
df[['main_category_fr','categories_fr']].sample(5)

,main_category_fr,categories_fr
119470,NaN,NaN
306595,NaN,NaN
102330,NaN,NaN
233872,NaN,NaN
14267,NaN,NaN


In [81]:
df[['brands_tags','product_name']].sample(5)

,brands_tags,product_name
33602,food-club,Pudding Snacks
281735,little-miracles,Roiibos Tea Elderberry Ginger Ginseng Agave
117575,"southern-home,bruno-s-supermarkets-inc","Candy, Peppermint Twist"
276418,boni,Salami
6523,roundy-s,Ice Cream


In [82]:
df[['nutrition_grade_fr']].sample(5)

,nutrition_grade_fr
311492,e
89021,e
291598,NaN
177807,e
148465,c


In [83]:
df.loc[:,var_basicPer100g].sample(5)

,calcium_100g,carbohydrates_100g,cholesterol_100g,energy_100g,fat_100g,fiber_100g,proteins_100g,salt_100g,saturated-fat_100g,sodium_100g,sugars_100g,trans-fat_100g
167964,NaN,0.00,0.117,2230.0,46.67,NaN,23.33,3.98018,20.0,1.567,0.00,0.0
129267,0.067,6.67,0.000,138.0,0.00,3.3,0.00,1.27000,0.0,0.500,3.33,0.0
49689,0.267,10.00,0.033,556.0,6.67,0.0,6.67,0.29718,5.0,0.117,10.00,0.0
1772,NaN,11.58,NaN,188.0,0.00,NaN,0.00,0.04318,NaN,0.017,11.58,NaN
146866,NaN,5.00,NaN,84.0,0.00,NaN,0.00,0.01270,NaN,0.005,5.00,NaN


# Notes:

68% products have the `nutrition_grade_fr` and `nutrition-score-fr_100g`: check this out